# Parseando páginas web: trabajar con el HTML

Dentro de la cadena del código HTML podemos buscar la información que nos interese recuperar. Podemos hacerlo de múltiples maneras. Empezaremos con algunas muy básicas y rudimentarias e iremos poco a poco mejorando la manera de acceder.

## Opción 1: Buscaremos a lo "bestia" con las funciones de búsqueda de cadenas básicas: str.find()

https://www.journaldev.com/23666/python-string-find

Revisando el código de la página encontramos la fecha de actualización de la misma:

    ...
    <span style="font-size:0.9vw"> [Actualizado a <span id="fecha">10 de Marzo de 2020</span> a las <span id="hora">10:10</span> horas]<br>Basada en  la notificación diaria de casos  agregados de COVID-19 al Ministerio de Sanidad
    ...
    
Y nos percatamos que la fecha en formato texto va encerrada entre: ```<span id="fecha">``` y ```</span>```. Busquémosla


In [ ]:
import requests                                         # Estas lineas las hemos copiado del anterior Notebook
respuesta = requests.get("https://zerjio.com/temp/covid19.html")  # Esta página reemplaza la anterior
html = respuesta.content.decode("UTF-8")                #    https://covid19.isciii.es/


cadenaBuscada = '<span id="fecha">'
posicionInicio = html.find(cadenaBuscada) + len(cadenaBuscada)
posicionFinal = html.find('</span>', posicionInicio)    # El segundo parámetro de find() es la posición desde
                                                        # donde empezar a buscar el primer parámetro

fecha = html[posicionInicio:posicionFinal]

print(fecha)

# Vamos a hacer lo mismo con la hora

cadenaBuscada = '<span id="hora">'
posicionInicio = html.find(cadenaBuscada) + len(cadenaBuscada)
posicionFinal = html.find('</span>', posicionInicio)

hora = html[posicionInicio:posicionFinal]

print(hora)

## Usando expresiones regulares

Las expresiones regulares nos permiten mucha expresividad para localizar elementos dentro de una cadena. Sin embargo en ocasiones su uso puede ser complejo en un documento HTML. Información interesante sobre las expresiones regulares:

https://docs.python.org/3/library/re.html

https://www.w3schools.com/python/python_regex.asp

https://www.guru99.com/python-regular-expressions-complete-tutorial.html

https://www.programiz.com/python-programming/regex

Herramienta online para testear expresiones regulares (seleccionar lenguaje Python):

https://regex101.com/

In [ ]:
# Recuperemos la fecha y hora con expresiones regulares.
# Usaremos el nuevo módulo "regex":
#   > conda install -c conda-forge regex

import regex as re

fecha = re.search(r'<span id=\"fecha\">(.*)</span>', html).group(1)
hora = re.search(r'<span id="hora">(.*)</span>', html).group(1)

print("%s - %s" % (fecha, hora))


In [ ]:
# Cuidado con las busquedas "exhaustivas". 
# Por ejemplo, podemos estar tentados a hacer una búsqueda de todas las fechas, por si hay más de una:

fecha = re.search(r'<span id="fecha">(.*)</span>', html, re.DOTALL).group(1)

print(fecha)

## Usando un parser SAX

_**S**imple **A**pi para **X**ml_ es una libreria basada en eventos: se va leyendo el archivo y se generan llamadas a funciones conforme se leen las etiquetas. Una de sus ventajas frente a otras soluciones es que no necesita cargar la fuente (el XML) completamente en memoria, lo cual lo hace idoneo para parsear ficheros MUY grandes. Sin embargo, puede ser tedioso y complejo de usar, como se verá en el ejemplo de más abajo



In [ ]:
# En este ejemplo vamos a usar el etree sax parser para obtener la información de la tabla que hay en la página 
# sobre 
from lxml import etree
from io import StringIO, BytesIO
import numpy as np
import matplotlib.pyplot as plt


class ParseCOVID():             # Manejadores de eventos
    tablaComenzada = False
    
    ultimaEtiquetaLeida = ""
    
    datosColumnas = []
    
    columnaActual = 0
    filaActual = -1
    
    def __init__(self):           # Este método se ejecuta al principio de parsear el documento
        return

    def start(self, tag, attrib): # Este método se ejecuta cuando aparece la apertura de una etiqueta
        if (tag == '{http://www.w3.org/1999/xhtml}th'):
            self.datosColumnas += [[]]  # inicializamos el array de cada columna
        
                    
        elif (tag == '{http://www.w3.org/1999/xhtml}tr') and (self.tablaComenzada == True):
            self.columnaActual = -1  # Empezamos a leer la siguiente fila
            self.filaActual += 1
            
        elif (tag == '{http://www.w3.org/1999/xhtml}td') and (self.tablaComenzada == True):
            self.columnaActual += 1
            self.datosColumnas[self.columnaActual].append("")  # Inicializamos el dato nuevo
            
        self.ultimaEtiquetaLeida = tag

    def end(self, tag):           # Este método se ejecuta cuando aparece el cierre de una etiqueta
        if (tag == '{http://www.w3.org/1999/xhtml}thead'):
            self.tablaComenzada = True



    def data(self, data):         # Este método se ejecuta con el contenido de una etiqueta
        if (self.ultimaEtiquetaLeida == '{http://www.w3.org/1999/xhtml}td') and (self.tablaComenzada == True):
            self.datosColumnas[self.columnaActual][self.filaActual] += data


    def close(self):              # Este método se ejecuta al final del archivo
        return
        
        
    def getColumns(self):    
        return self.datosColumnas
    
# Creamos el objeto analizador (de la clase de más arriba), creamos un parser con él y le decimos 
# a la biblioteca lmxl.etree que parsee el documento HTML con este tipo de parser SAX
analizador = ParseCOVID()
parser = etree.XMLParser(target=analizador, recover=True)
etree.parse(StringIO(html), parser)


# Obtenemos la información sacada de la tabla
datosColumnas = analizador.getColumns()
comunidades = datosColumnas[0]
casos = datosColumnas[1]

# Quitamos caracteres extraños a los nombres de las comunidades
comunidades[:] = [n.strip() for n in comunidades]

# Convertimos a entero los datos numericos (quitando caracteres extraños)
casos[:] = [int(d.strip()) for d in casos]


# Creamos el gráfico de barras
plt.bar(comunidades, casos)
# Rotation of the bars names
plt.xticks(rotation=90)
# Show graphic
plt.show()


## Usando un parser DOM

_**D**ocument **O**bject **M**odel_ es una interfaz que permite manejar ficheros *XML* (y derivados, como *HTML*) mediante objetos que se encuentran en memoria formando una estructura de árbol (replicando la estructura de las etiquetas). Las interfaces *DOM* son normalmente mucho más cómodas e intuitivas de usar que *SAX*, aunque no funcionan bien para ficheros muy grandes. A continuación veremos unos pocos ejemplos de bibliotecas y herramientas que funcionan con este tipo de interfaz.

### MiniDOM
Ejemplo de uso de minidom (no es bueno para HTML que suele no estar bien formado, solo para XML). Por eso vamos a usar un fichero RSS de noticias

Documentacion de la biblioteca: https://docs.python.org/3.3/library/xml.dom.minidom.html

In [ ]:
import xml.dom.minidom

url = 'http://ep00.epimg.net/rss/elpais/portada.xml'
rss = requests.get(url).content.decode("UTF-8")

dom = xml.dom.minidom.parseString(rss)

# Obtenemos todas las etiquetas "item"
items = dom.getElementsByTagName("item")

# Para cada etiqueta mostraremos el título de la noticia asociada y la url de la imagen asociada (si la tiene)
for item in items:
    titulo = item.getElementsByTagName("title")
    
    print(" + " + titulo[0].firstChild.nodeValue)   # Detalle "peliagudo", el texto de dentro de una etiqueta 
                                                    # es considerado un hijo. De hecho una etiqueta puede tener
                                                    # varios hijos de "texto". Ejemplo:
                                                    # <etiquetaConTexto>
                                                    #   Esto es texto
                                                    #   <etiquetaHijo />
                                                    #   Más texto
                                                    # </etiquetaConTexto>
                                                    #
                                                    # El elemento "etiquetaConTexto" tiene tres hijos:
                                                    #    + Hijo 1: de tipo texto: "\n  Esto es texto\n  "
                                                    #    + Hijo 2: etiqueta "etiquetaHijo"
                                                    #    + Hijo 3: de tipo texto: "\n  Más text\n  "

# En este ejemplo, las urls de las imágenes asociadas a un item están en la etiqueta "enclosure", atributo "url":

    enclosures = item.getElementsByTagName("enclosure")
    
    for e in enclosures:
        print("   - " + e.getAttribute('url'))

### DOM parser lxml.etree

Esta *API* es más sencilla y está más adaptada a Python:

+ Los elementos son listas (con lo que podemos acceder a sus hijos por índices)
+ Permite fácilmente modificar los elementos del *DOM* (lo mismo no es muy útil si estamos solo obteniendo información, pero vete tu a saber).
+ Los atributos de un elemento son un diccionario que está en elemento.attrib, con lo que acceder a ellos es muy "natural".


Tutorial: https://lxml.de/tutorial.html

In [ ]:
# Repetimos el ejercicio anterior pero con el parser DOM lxml.etree
from lxml import etree

tree = etree.parse(url)

raiz = tree.getroot()

# Iteramos por todos los descendientes de raiz que sean "item"

for item in raiz.iter("item"):
    for hijo in item.iter("title"):
        print(" + " + hijo.text)
    
    for hijo in item.iter("enclosure"):
        print("   - " + hijo.attrib['url'])
    

### Usando _XPath_ para localizar elementos fácilmente

*XPath* (*XML Path Language*) es un lenguaje que permite construir expresiones que recorren y procesan un documento *XML*. Permite buscar y seleccionar teniendo en cuenta la estructura jerárquica del *XML*.

+ https://es.wikipedia.org/wiki/XPath
+ https://www.w3schools.com/xml/xpath_syntax.asp

In [ ]:
# Replicaremos el ejemplo de la gráfica de casos de COVID-19 de manera mucho más simple
from lxml import etree
from io import StringIO, BytesIO
import matplotlib.pyplot as plt

parserHTML = etree.HTMLParser()

# Recuerda que html se inicializó mucho antes con el contenido de la página https://covid19.isciii.es/
tree = etree.parse(StringIO(html), parserHTML)

tablas = tree.xpath('//table')  # Nos devuelve todos los elementos "table" en cualquier sitio del documento
                                # En nuestro caso habrá devuelto 2 tablas

# Nos interesa la segunda tabla de la página 
tabla = tablas[1]

comunidades = []
casos = []

filas = tabla.xpath('tbody/tr')  # Buscamos todos los <tr> que estén dentro de <tbody> en la tabla

for f in filas:
    comunidades.append(f[0].text)
    casos.append(int(f[1].text))
    
    
# Creamos el gráfico de barras
plt.bar(comunidades, casos)
# Rotation of the bars names
plt.xticks(rotation=90)
# Show graphic
plt.show()